## Class: AIT 526                                             September 15, 2021
### Professor:  Dr. Duoduo (Lindi) Liao
### Team 5: 
- Anh "Tim" Hien Bach
- Robert "Robb" Jay Dunlap
- Vishnu Lasya Marthala
- David Earl Swanson

#### Programming Assignment – Chatbot Eliza
Write an Eliza program in Python. No chat packages/functions/libraries are allowed to use. The program should be called eliza.py, and it should run from the command line with no arguments. NOTE: if you use **jupyter notebook**, you can have all comments and required running outputs/logs in the notebook file(s). And then save into HTML(s) and zip all notebook file(s), HTML files, and other files into ONE zip file. Please submit only one zip file.

Your program should engage in a dialogue with the user, with your program Eliza playing the role of a psychotherapist. Your program should be able carry out "word spotting", that is it should recognize certain key words and respond simply based on that word being present in the input. It should also be able to transform certain simple sentence forms from statements (from the user) into questions (that Eliza will ask). Also, try to personalize the dialogue by asking and using the user's name. 

In addition, your program should be robust. If the user inputs gibberish or a very complicated question, Eliza should respond in some plausible way (I didn't quite understand, can you say that another way, etc.). “Word spotting”, sentence transformation, and robustness are the minimum requirements for your code. You can implement additional functionalities, inspired by the dialogues presented in Weizenbaum paper. You may receive up to 1 bonus point max for any additional functionalities. 

This program should rely heavily on the use of regular expressions, so please make sure to review some introductory material in Learning Python, Programming Python, or some other source before attempting this program.

 

#### Tasks
- 

### Be sure to comment your code. In particular, explain what words you are spotting for (and why) and what statement forms you are converting into questions (and why). Also make sure your name, class, etc. is clearly included in the comments.

Due: September 21, 2021

### ELIZA

***

### Load libraries 

In [1]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast
import nltk 
import re
from nltk.corpus import wordnet 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import random
import sys
import os
#import txt 

from nltk import pos_tag
from nltk import RegexpParser, RegexpChunkParser
from nltk.tokenize import word_tokenize
from string import punctuation
#from nltk.stem import WordNetLemmatizer
#import contractions

# Using the TreebankTagger instead of the Perceptron model in NLTK (TreebankTagger is muuch better)
# I found an explanation and the below line of code in this SO post:

# https://stackoverflow.com/questions/30821188/python-nltk-pos-tag-not-returning-the-correct-part-of-speech-tag
treebankTagger = nltk.data.load('taggers/maxent_treebank_pos_tagger/english.pickle')
os.chdir('D:\\Fall 2021\\NLP- AIT 526\\Programmming Assignments\\1. Prg Ass')
os.getcwd()
#vishnu 


'D:\\Fall 2021\\NLP- AIT 526\\Programmming Assignments\\1. Prg Ass'

### We decided to "load" or read in several dictionaries that support the algorithm in each function rather than hard coding them into the notebook.
####    This section loads those dictionaries
Code written by:  The full team

In [2]:
# Read in a dictionary of contractions
file = open("contraction_dictionary.txt", "r")
contents = file.read()
contractions = ast.literal_eval(contents)
file.close

# Read in a dictionary of the emergency words
file = open("emergency_words.txt", "r")
content = file.read()
emergency_words = content.split(",")
file.close

# Read in the emergency response
file = open('emergency_response.txt', 'r', encoding = 'utf-8')
emergency_response = file.read()
file.close

# Open the nonspecific responses that will be used when Eliza doesn't understand the patient's statement.
file = open("gibberish_responses.txt", "r")
nonspecific_response = file.readlines()
file.close

# Open the list of words to be spotted and find synonyms of them to creaste a dictionary of spotting words
file = open("spotted_words.txt", "r")
list_key = file.read()
file.close
#vishnu
file = open("Standard_Responses.txt", "r")
standardresponse = file.read()
standard_response=ast.literal_eval(standardresponse)
file.close
print(standard_response)
#print((standard_response['sad']))

#vishnu
file = open("reg_exp_pattern.txt", "r")
reg_exp_pattern = file.read()
reg_exp__pattern=ast.literal_eval(reg_exp_pattern)
file.close
print(reg_exp__pattern)


dic_spotwords={}
ListKey = list_key.split()
print(ListKey)
for word in ListKey:
    dup_synonyms=[]
    for synonym in wordnet.synsets(word):
        for lemma in synonym.lemmas():
            dup_synonyms.append(lemma.name())

    unique_synonyms=[]
    for synonym in dup_synonyms:
        if synonym not in unique_synonyms:
            unique_synonyms.append(synonym)
    dic_spotwords[word]=unique_synonyms
print(dic_spotwords)
# I don't think we use these regular expressions
#remove reg exp dictionary #vishnu

#These need to be greatly ezxpanded and turnedinto a file for reading
#vishnu removed those disctonary (standard responses)

# This section loads the necessary information for the "alternate_phrase" function below. The csv contains:
# (1) RE patterns for creating specific chunks from using POS labels
# (2) Identifying the portion of the user input that will be "turned" back to them in the response
# (3) The framework response phrases that the portion identifiied in (2) will be married to

input_terms_df = pd.read_csv('input_chunk_terms.csv', 
                             dtype={'capture_expression':'str', 
                                     'insertion':'bool', 
                                     'response_template':'str'}
                            )

{'greet': 'Cool! Go ahead.', 'help': 'I am here to help you with your emotions and thought process.', 'yes': 'Tell me more about it.', 'happy': 'Awesome! What makes you happy?', 'good': 'Fab! What makes you feel good?', 'bad': 'Oh! But, What makes you feel bad?', 'sick': 'What makes you sick?', 'healthy': 'What makes you feel healthy?', 'worst': 'What makes you feel worst?', 'unhealthy': 'What makes you feel unhealthy?', 'best': 'What makes you feel the best?', 'bye': 'Bye!', 'discourage': 'What kind of attitude do you have? Is it Positive or Negative?', 'motiveless': 'What kind of attitude do you have? Is it Positive or Negative?', 'positive': 'Cool! Go ahead.', 'negative': 'Try to read motivational books and also listen to podcasts for positive vibes.', 'support': 'Change your thoughts and put your plans to action, and I can assure, you are going to feel much better!', 'grateful': 'Happy to help! Be Cool and Stay Positive.', 'appreciate': 'Happy to help! Be Cool and Stay Positive.', 

### Functions used by the main program

In [3]:
# Code written by Vishnu Lasya Marthala and edited by David Swanson 
#vishnu  
def standard_phrase(user_keyword):
    
    #print('into the loop')
    for key_r,value_r in standard_response.items():
        #print(key_r)
        if(user_keyword ==  key_r ):
            #print('userkeyword:',user_keyword,key_r)
            return(value_r)
    return("")



In [4]:
# remove #vishnu 
#def standard_phrase(text):
    #for token in text:    
    # for key, value in dic_spotwords.items():
           # if text in value:
                #print('text is', text)
              #  for key_r,value_r in standardResponses.items():
                   # print(key_r)
                   # if(key == key_r):
                      #  print('went into final if')
                      #  return(value_r)
           # return("")
#vishnu
def regex_pattern(sentence):
    for key,value in reg_exp__pattern.items():
        m=re.search(key,sentence.lower())
        ta=None
        if(m):
            s=random.choice(value)
            for rpy in s:
                if '{0}' in s:
                    ta=m.group(1)
                    return(s.format(ta))
    return("")

In [5]:
# Alternative method for parsing user input and generating a response - code written by Robb Dunlap

def alternate_phrase(user_sentence):
    new_sentence = ""
    new_word = ""
    for word in user_sentence.split():
        for key, root in contractions.items():
            if key == word:
                word = root
        new_word = word+" "
        new_word += " "
        # Had to put the words back together as a string instead of as a list because the two separate base words on the contraction
        # end up being one entry in a list. So instead, reassemble the sentence as a string and split it again to be sure to get
        # individual words
        new_sentence += new_word

    # need to strip off extra space added to the end of the sentence
    new_sentence = new_sentence[:-2]

    # create a list of the sentence words in order
    split_sent = new_sentence.split()
    sent_words_wo_punct = [w.strip(punctuation) for w in split_sent]


    tokens_tag = treebankTagger.tag(sent_words_wo_punct)

    index_in_df = -1
    # for statement in chunk_list:
    for statement in input_terms_df['capture_expression']:
        pattern_found = 0
        index_in_df+=1
        phrase_pattern = statement
        chunker = RegexpParser(phrase_pattern)
        if index_in_df<2:
            output = chunker.parse(tokens_tag)
        else:
            output = chunker.parse(output)

        key_word = []
        for item in output:
            if isinstance(item, nltk.tree.Tree):
                pattern_found = 1
                regex_desired_word = re.compile(input_terms_df.iloc[index_in_df,3])
                for thing in item:
                    if re.match(regex_desired_word, thing[1]):     
                        key_word = thing[0]
        if pattern_found == 1:
            break

    # the below section captures the desired portion of the sentence so it can be flipped into the response

    position_in_sent_word = 0
    counter = 0
    for element in sent_words_wo_punct:
        if key_word == element:
            counter = position_in_sent_word
        position_in_sent_word += 1

    offset_in_sentence = counter + input_terms_df.iloc[index_in_df,4]
    turned_portion_of_sent = sent_words_wo_punct[offset_in_sentence:]

    # This section appends or inserts the flipped portion of the input text to the appropriate template 
    if pattern_found == 1:
        if not input_terms_df.iloc[index_in_df,1]:
            offset_in_sentence = counter + input_terms_df.iloc[index_in_df,4]
            turned_portion_of_sent = sent_words_wo_punct[offset_in_sentence:]
            temp_holder = ""
            separator = " "
            temp_holder = separator.join(turned_portion_of_sent)
            question_mark = "?"
            temp_holder.join(question_mark)
            eliza_response = input_terms_df.iloc[index_in_df,2]+" "+temp_holder+"?"
        else:
            offset_in_sentence = counter + input_terms_df.iloc[index_in_df,4]
            turned_portion_of_sent = sent_words_wo_punct[offset_in_sentence:]
            temp_holder = ""
            separator = " "
            temp_holder = separator.join(turned_portion_of_sent)
            turned_portion_of_sent = str(temp_holder)
            eliza_response = input_terms_df.iloc[index_in_df,2].replace("XXYYMM",turned_portion_of_sent)
        return eliza_response    

    else:
        elize_response = ""
        return elize_response

### Eliza welcomes the patient, sets the tone for the conversation, and establishes an end of session codeword "quit."
#### We decided that Eliza could asked for the patient's name which is used to tag the patient's inputs lines as well as a salutation.
Code written by David Swanson and Vishnu Lasya Marthala

**To use Spacy en_core_web_sm** I had to load add the Spacy module to my Conda environment and then I also had to use the following code in a terminal in my environment to add the library: &emsp; `conda install -c conda-forge spacy-model-en_core_web_sm`


#### The following code is a loop that
1. Tokenizes the sentances in the response
2. If the patient talks too long (more than two sentences) Eliza asks in several ways for the patient to slow down 
3. Tokenizes the words of each sentence to be sent to four functions in sequence.
    1. Has the patient made a statement that is dangerous to themselves of the theoropist (aka emergency words)? 
                     - If so, instruct the patient to seek help and end the session 
    2. Can Eliza respond using spotted words?
    3. Can a sophisticated regular expression algorithm make a response?
    4. If all these fail, Eliza assumes the answer was gibberish and randomly selects a response for the patient to talk more.
    
Code written by: David Swanson

In [ ]:
# print("Eliza: Hello! My name is Eliza.  Welcome to my practice.")
print("Eliza: I don't want you to think of these sessions as therapy but rather as an opportunity for self-reflection and growth.")
print('Eliza: As this is a safe place, you can stop our session at anytime by typing "Quit."')
print("")

# Get the patient's name
print("Eliza: How can I address you?") #vishnu
print("")
user_input = input("Patient: ")

nlp = spacy.load("en_core_web_sm")

doc = nlp(user_input)  
print(doc.ents )
usernames=[e for e in doc.ents if e.label_ == 'PERSON']


if doc.ents==():
    patientName = "Patient: "
    print("Eliza: That's ok, we don't need to use names. So how are you feeling today?")
else:
    patientName=usernames[0].text
    print('Eliza: Hello '+ patientName +'. '+"How are you feeling today?")

    
# Entering while loop for continuing discussion with the patient

loopAgain = True

# Top of the loop
while (loopAgain == True):
    
    # Get the patient's input 
    print("")
    user_input = input(patientName+": ")
    
# First check the patient's response to see if they want to end the session.
    if (user_input.lower() == "quit"):
        print("Eliza: I hope our session was helpful.  Goodbye",patientName)
        loopAgain = False
        break
        
# Start a counter to cycle through some random responses when the patients drones on
    lengthyReponse = 0
    
# Separate the response into individual sentences
    PatientResponse = nltk.sent_tokenize(user_input)
    # print(PatientResponse)
    
# Send the patient's response word by word to check for emergency words
# Code written by Anh "Tim" Hien Bach

    HaltFlag = 0
    for sentences in PatientResponse:
        PatientWords = nltk.word_tokenize(sentences)
        for words in PatientWords: 
            for EmWord in emergency_words:
                # If any emergency word is found, return with a standard cautionary response
                if re.search(EmWord, words):
                    print(patientName,": ", emergency_response)
                    loopAgain = False
                    break
            if loopAgain == False:
                break
        if loopAgain == False:
            break

# If the response is more than 2 sentences
    if loopAgain == True and len(PatientResponse)>2:
        lengthyReponse = lengthyReponse + 1

# Cycle through two responses if the patient inputs 3 sentences 
        if(len(PatientResponse)==3): 
            if(lengthyReponse == 1):
                print("Eliza: Let's go slower and take things one at a time. What is on your mind?")
            if(lengthyReponse > 1):
                print("Eliza: Again, slow down. Try again.")
                lengthy_response = 0
                
# Cycle through three three responses if the patient inputs more than 3 sentences 
        else: 
            if(lengthyReponse == 1):
                print("Eliza: Whoa, that's a lot to unpack. Let's try again with shorter thoughts.")
            if(lengthyReponse == 2):
                print("Eliza: Again, slow down. Try again.")
            if(lengthyReponse > 2):
                print("Eliza: You have alot on your mind. Let's start again with your first thought.")
                lengthyReponse = 0

# If the patient gives one or two sentence response, address each            
    else:
        if loopAgain == False:
            break
        for sentence in PatientResponse:
            
            # We don't weant contractions in our evaluation of the patient's input so we expand them 
            newSentence = ""
            for word in sentence.split():
                for key, root in contractions.items():
                    if key == word:
                        word = root
                    elif key == word.lower():
                        word = root
            
            # Tokenize the updated sentence into words
            tokenizer = RegexpTokenizer (r'\w+')
            words_only = tokenizer.tokenize(sentence)
            #print(words_only)
            # Debugging Code
            # print ("The total number of words in sentence", i, "of the user's input is:", len(words_only)) 
            # print (words_only)
    
            # Set ElizaResponse to NULL
            ElizaResponse = ""
            
            #vishnu
            # Send the patient's sentence to a word spotter routine 
            match =0 
            for token in words_only:
                for key, value in dic_spotwords.items():
                    #print('key is : ', key)
                    if token in value:
                        #print(token)
                        match=1
                        #if ElizaResponse != "":
                        ElizaResponse = standard_phrase(key)
                        print("Eliza1: ", ElizaResponse)
                        break
                        
            #vishnu     
            if(match!=1):
                ElizaResponse = regex_pattern(sentence)
                if(ElizaResponse != ""):
                    match=2
                    
                    print("Eliza2: ", ElizaResponse)
                    break
                               
                
            # If the standard phrase routine fails to provide a response send to the alternate function 
            # We use the untokenized
            if(match!=1 and match!=2): #vishnu
                ElizaResponse = alternate_phrase(PatientResponse[0])
                if ElizaResponse != "":
                    match=3
                    print("Eliza3: ", ElizaResponse)
                    break
                    
            # Those don't provide a response, it is gibberish so send a random response
            if(match!=1 and match !=2 and match!=3): #vishnu
                print("Eliza4: ", random.choice(nonspecific_response))

Eliza: I don't want you to think of these sessions as therapy but rather as an opportunity for self-reflection and growth.
Eliza: As this is a safe place, you can stop our session at anytime by typing "Quit."

Eliza: How can I address you?

Patient: I am Stella
(Stella,)
[Stella]
Eliza: Hello Stella. How are you feeling today?

Stella: can you help me
Eliza1:  I am here to help you with your emotions and thought process.

Stella: okay. i want to know more
Eliza4:  That is interesting, go on.

Eliza4:  Tell me more.


Stella: okay. i am sad
Eliza4:  Please tell me more.

Eliza2:  Why do you think you are sad?

Stella: i am not sure
Eliza2:  Why do you think you are not sure?

Stella: but, i am positive
Eliza1:  Cool! Go ahead.

Stella: i am having wonderful time
Eliza2:  Why do you think you are having wonderful time?

Stella: i am not sure
Eliza2:  Why do you think you are not sure?

Stella: i crave to have more in life
Eliza2:  Why do you crave have more in life?

Stella: do you care


In [53]:
PatientResponse

['i am sad']